## Importing libraries


In [1]:
import time
from copy import deepcopy

import numpy as np
import torch
from sklearn.metrics import (
    accuracy_score,
    cohen_kappa_score,
    confusion_matrix,
    mean_absolute_error,
)
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.utils import class_weight
from torch import cuda
from torch.optim import Adam
from torch.utils.data import DataLoader, Subset
from torchvision.models import resnet18
from torchvision.transforms import Compose, ToTensor
from tqdm import tqdm

from dlordinal.datasets import FGNet
from dlordinal.losses import OrdinalECOCDistanceLoss
from dlordinal.wrappers import OBDECOCModel

## Load and preprocess of FGNet dataset

First, we present the configuration parameters for the experimentation and the number of workers for the *DataLoader*, which defines the number of subprocesses to use for data loading. In this specific case, it refers to the images.

In [2]:
optimiser_params = {"lr": 1e-3, "bs": 200, "epochs": 5, "s": 2, "c": 0.2, "beta": 0.5}

workers = 3

Now we use the *FGNet* method to download and preprocess the images. Once that is done with the training data, we create a validation partition comprising 15% of the data using the *StratifiedShuffleSplit* method. Finally, with all the partitions, we load the images using a method called *DataLoader*.

In [5]:
fgnet_trainval = FGNet(
    root="./datasets",
    download=True,
    train=True,
    target_transform=np.array,
    transform=Compose([ToTensor()]),
)

test_data = FGNet(
    root="./datasets",
    download=True,
    train=False,
    target_transform=np.array,
    transform=Compose([ToTensor()]),
)

num_classes = len(fgnet_trainval.classes)
classes = fgnet_trainval.classes
targets = fgnet_trainval.targets

# Create a validation split
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.15, random_state=0)
sss_splits = list(sss.split(X=np.zeros(len(fgnet_trainval)), y=fgnet_trainval.targets))
train_idx, val_idx = sss_splits[0]

# Create subsets for training and validation
train_data = Subset(fgnet_trainval, train_idx)
val_data = Subset(fgnet_trainval, val_idx)

# Get CUDA device
device = "cuda" if cuda.is_available() else "cpu"
print(f"Using {device} device")

# Create dataloaders
train_dataloader = DataLoader(
    train_data, batch_size=optimiser_params["bs"], shuffle=True, num_workers=workers
)
val_dataloader = DataLoader(
    val_data, batch_size=optimiser_params["bs"], shuffle=True, num_workers=workers
)
test_dataloader = DataLoader(
    test_data, batch_size=optimiser_params["bs"], shuffle=False, num_workers=workers
)

# Get image shape
img_shape = None
for X, _ in train_dataloader:
    img_shape = list(X.shape[1:])
    break
print(f"Detected image shape: {img_shape}")

# Define class weights for imbalanced datasets
classes_array = np.array([int(c) for c in classes])

class_weights = (
    torch.from_numpy(
        class_weight.compute_class_weight("balanced", classes=classes_array, y=targets)
    )
    .float()
    .to(device)
)
print(f"{class_weights=}")

Files already downloaded and verified
Files already processed and verified
Files already split and verified
Files already downloaded and verified
Files already processed and verified
Files already split and verified
Using cpu device


Detected image shape: [3, 128, 128]
class_weights=tensor([1.0191, 1.5345, 0.7946, 1.1314, 0.5517, 2.4273])


## Model and optimiser

We are using a modified version of the *ResNet* architecture, specifically designed for the loss function explained in the next section. In this case, the *ResNet* model is not pretrained with *ImageNet*, so we will need to undergo an extensive learning process.

To adapt the outputs of the model to this, the final fully-connected block is substituted by $Q-1$ fully-connected blocks [1], each one with a single output unit with sigmoid activation.

As an alternative to the *ResNet* architecture, you can use the *VGG* architecture, which has also been implemented to work with the loss function explained in the following sections.

Finally, we define the *Adam* optimiser, which is used to adjust the network's weights and minimize the error of a loss function.

[1]: Barbero-Gómez, J., Gutiérrez, P. A., & Hervás-Martínez, C. (2022). *Error-correcting output codes in the framework of deep ordinal classification.* Neural Processing Letters, 1-32. Springer.

In [6]:
model = OBDECOCModel(num_classes, resnet18(num_classes=1000), base_n_outputs=1000).to(
    device
)

# Optimizer
optimizer = Adam(model.parameters(), lr=optimiser_params["lr"])

## Loss Function

The original $Q$-class ordinal problem is decomposed into $Q-1$ binary decision problems, what is known as Ordinal Binary Decomposition (ODB) [1]. So the categorical cross-entropy has been substituted by the Mean Squared Error loss because it copes better with the distance function used for the Error-Conecting Output Codes (ECOC) decision:

$$
\ell = \frac{1}{N} ∑_{i=1}^N ∑_{k=1}^{Q-1} (\mathbf{1} \{y_i \succ \mathcal{C}_k\} - P(y_i \succ \mathcal{C}_k | x_i))^2
$$

where $\mathbf{1} \{y_i \succ \mathcal{C}_k\}$ is the indicator function that is equal to 1 when $y_i \succ \mathcal{C}_k$ and 0 otherwise, and $P(y_i \succ \mathcal{C}_k | x_i)$ is the probability that $y_i \succ \mathcal{C}_k$ predicted by the network given a sample.

[1]: Barbero-Gómez, J., Gutiérrez, P. A., & Hervás-Martínez, C. (2022). *Error-correcting output codes in the framework of deep ordinal classification.* Neural Processing Letters, 1-32. Springer. doi.org/10.1007/s11063-022-10824-7

In [7]:
loss_fn = OrdinalECOCDistanceLoss(num_classes=num_classes, weights=class_weights).to(
    device
)

## Metrics

In [8]:
# Metrics computation


def compute_metrics(y_true: np.ndarray, y_pred: np.ndarray, num_classes: int):

    if len(y_true.shape) > 1:
        y_true = np.argmax(y_true, axis=1)

    if len(y_pred.shape) > 1:
        y_pred = np.argmax(y_pred, axis=1)

    labels = range(0, num_classes)

    # Metrics calculation
    qwk = cohen_kappa_score(y_true, y_pred, weights="quadratic", labels=labels)
    ms = minimum_sensitivity(y_true, y_pred, labels=labels)
    mae = mean_absolute_error(y_true, y_pred)
    acc = accuracy_score(y_true, y_pred)
    off1 = accuracy_off1(y_true, y_pred, labels=labels)
    conf_mat = confusion_matrix(y_true, y_pred, labels=labels)

    metrics = {
        "QWK": qwk,
        "MS": ms,
        "MAE": mae,
        "CCR": acc,
        "1-off": off1,
        "Confusion matrix": conf_mat,
    }

    return metrics


def _compute_sensitivities(y_true, y_pred, labels=None):
    if len(y_true.shape) > 1:
        y_true = np.argmax(y_true, axis=1)
    if len(y_pred.shape) > 1:
        y_pred = np.argmax(y_pred, axis=1)

    conf_mat = confusion_matrix(y_true, y_pred, labels=labels)

    sum = np.sum(conf_mat, axis=1)
    mask = np.eye(conf_mat.shape[0], conf_mat.shape[1])
    correct = np.sum(conf_mat * mask, axis=1)
    sensitivities = correct / sum

    sensitivities = sensitivities[~np.isnan(sensitivities)]

    return sensitivities


def minimum_sensitivity(y_true, y_pred, labels=None):
    return np.min(_compute_sensitivities(y_true, y_pred, labels=labels))


def accuracy_off1(y_true, y_pred, labels=None):
    if len(y_true.shape) > 1:
        y_true = np.argmax(y_true, axis=1)
    if len(y_pred.shape) > 1:
        y_pred = np.argmax(y_pred, axis=1)

    conf_mat = confusion_matrix(y_true, y_pred, labels=labels)
    n = conf_mat.shape[0]
    mask = np.eye(n, n) + np.eye(n, n, k=1), +np.eye(n, n, k=-1)
    correct = mask * conf_mat

    return 1.0 * np.sum(correct) / np.sum(conf_mat)


def print_metrics(metrics):
    print("")
    print("Confusion matrix :\n{}".format(metrics["Confusion matrix"]))
    print("")
    print("MS: {:.4f}".format(metrics["MS"]))
    print("")
    print("QWK: {:.4f}".format(metrics["QWK"]))
    print("")
    print("MAE: {:.4f}".format(metrics["MAE"]))
    print("")
    print("CCR: {:.4f}".format(metrics["CCR"]))
    print("")
    print("1-off: {:.4f}".format(metrics["1-off"]))

## Training process

In [13]:
def train(
    dataloader: torch.utils.data.DataLoader,
    model: OBDECOCModel,
    loss_fn: torch.nn.Module,
    optimizer: torch.optim.Optimizer,
    device: torch.device,
    H: dict,
    num_classes: int,
):  # H: dict
    num_batches = len(dataloader)
    size = len(dataloader.dataset)
    progress_bar = tqdm(total=num_batches, ncols=100, position=0, desc="Train progress")
    model.train()
    mean_loss, accuracy = 0, 0
    y_pred, y_true = None, None

    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)  # Inputs and labels to device

        # Compute prediction error and accuracy of the training process
        pred = model(X)
        loss = loss_fn(pred, y)

        mean_loss += loss
        accuracy += (pred.argmax(1) == y).type(torch.float).sum().item()

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Stack predictions and true labels to determine the confusion matrix
        pred_np = model.transformer.labels(pred).cpu().detach().numpy()
        true_np = y.cpu().detach().numpy()
        if y_pred is None:
            y_pred = pred_np
        else:
            y_pred = np.concatenate((y_pred, pred_np))

        if y_true is None:
            y_true = true_np
        else:
            y_true = np.concatenate((y_true, true_np))

        # Update progress bar
        progress_bar.set_postfix(loss=loss.item(), accuracy=accuracy)
        progress_bar.update(1)

    accuracy /= size
    mean_loss /= num_batches

    H["train_loss"].append(loss.cpu().detach().numpy())
    H["train_acc"].append(accuracy)

    # Confusion matrix for training
    labels = range(0, num_classes)
    conf_mat = confusion_matrix(y_true, y_pred, labels=labels)
    print("")
    print("Train Confusion matrix :\n{}".format(conf_mat))
    print("")

    return accuracy, mean_loss

## Testing process

In [10]:
def test(
    test_dataloader: torch.utils.data.DataLoader,
    model: OBDECOCModel,
    loss_fn: torch.nn.Module,
    device: torch.device,
    num_classes: int,
):
    num_batches = len(test_dataloader)
    progress_bar = tqdm(total=num_batches, ncols=100, position=0, desc="Test progress")
    model.eval()
    test_loss = 0
    y_pred, y_true = None, None

    with torch.no_grad():
        for batch, (X, y) in enumerate(test_dataloader):
            X, y = X.to(device), y.to(device)  # inputs and labels to device
            pred = model(X)
            test_loss += loss_fn(pred, y).item()

            # Stack predictions and true labels
            pred_np = model.transformer.labels(pred).cpu().detach().numpy()
            true_np = y.cpu().detach().numpy()
            if y_pred is None:
                y_pred = pred_np
            else:
                y_pred = np.concatenate((y_pred, pred_np))

            if y_true is None:
                y_true = true_np
            else:
                y_true = np.concatenate((y_true, true_np))

            # Update progress bar
            progress_bar.set_postfix(loss=test_loss / (batch + 1))
            progress_bar.update(1)

    test_loss /= num_batches
    metrics = compute_metrics(y_true, y_pred, num_classes)
    print_metrics(metrics)

    return metrics, test_loss

## Validation process

In [11]:
def validate(
    dataloader: torch.utils.data.DataLoader,
    model: OBDECOCModel,
    loss_fn: torch.nn.Module,
    device: torch.device,
    H: dict,
    num_classes: int,
):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    loss, accuracy = 0, 0
    y_pred, y_true = None, None

    with torch.no_grad():
        for batch, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)
            pred = model(X)
            loss += loss_fn(pred, y)
            accuracy += (pred.argmax(1) == y).type(torch.float).sum().item()

            pred_np = model.transformer.labels(pred).cpu().detach().numpy()
            true_np = y.cpu().detach().numpy()
            if y_pred is None:
                y_pred = pred_np
            else:
                y_pred = np.concatenate((y_pred, pred_np))

            if y_true is None:
                y_true = true_np
            else:
                y_true = np.concatenate((y_true, true_np))

    accuracy /= size
    loss /= num_batches

    H["val_loss"].append(loss.cpu().detach().numpy())
    H["val_acc"].append(accuracy)

    metrics = compute_metrics(y_true, y_pred, num_classes)

    return metrics, accuracy, loss

## Results

In [14]:
H = {"train_loss": [], "train_acc": [], "val_loss": [], "val_acc": []}

# To store validation metrics
validation_metrics = {}

# Definition to store best model weights
best_model_weights = model.state_dict()
best_qwk = 0.0

# Start time
start_time = time.time()

for e in range(optimiser_params["epochs"]):
    train_acc, train_loss = train(
        train_dataloader, model, loss_fn, optimizer, device, H, num_classes=num_classes
    )
    validation_metrics, val_acc, val_loss = validate(
        val_dataloader, model, loss_fn, device, H, num_classes=num_classes
    )

    if validation_metrics["QWK"] >= best_qwk:
        best_qwk = validation_metrics["QWK"]
        best_model_weights = deepcopy(model.state_dict())

    print("[INFO] EPOCH: {}/{}".format(e + 1, optimiser_params["epochs"]))
    print("Train loss: {:.6f}, Train accuracy: {:.4f}".format(train_loss, train_acc))
    print("Val loss: {:.6f}, Val accuracy: {:.4f}\n".format(val_loss, val_acc))

# Store last train error
train_error = H["train_loss"][-1]

# Restore best weights
model.load_state_dict(best_model_weights)

# Start evaluation
print("[INFO] Network evaluation ...")

test_metrics, test_loss = test(
    test_dataloader, model, loss_fn, device, num_classes=num_classes
)

# End time
end_time = time.time()
print("\n[INFO] Total training time: {:.2f}s".format(end_time - start_time))

Train progress: 100%|█████████████████████████| 4/4 [00:11<00:00,  2.90s/it, accuracy=74, loss=39.1]


Train Confusion matrix :
[[  0  71   2   1   0   0]
 [  0 175  16  13   0   1]
 [  0  44  25  32   3   7]
 [  0  19  21  81   9  13]
 [  0  11   8  35   7  39]
 [  0   0   0   6   3  38]]



[INFO] EPOCH: 1/5
Train loss: 86.180344, Train accuracy: 0.1088
Val loss: 138.933105, Val accuracy: 0.1074



Train progress: 100%|█████████████████████████| 4/4 [00:10<00:00,  2.69s/it, accuracy=74, loss=31.9]


Train Confusion matrix :
[[  0  70   2   2   0   0]
 [  0 181  16   8   0   0]
 [  0  44  25  37   1   4]
 [  0  14  17  99   3  10]
 [  0   5   9  59  11  16]
 [  0   0   0   1   3  43]]



...

Train progress: 100%|█████████████████████████| 4/4 [00:11<00:00,  2.78s/it, accuracy=74, loss=22.5]


Train Confusion matrix :
[[  0  74   0   0   0   0]
 [  0 196   7   2   0   0]
 [  0  23  49  39   0   0]
 [  0   3   9 131   0   0]
 [  0   1   1  49  31  18]
 [  0   0   0   0   0  47]]



[INFO] EPOCH: 5/5
Train loss: 44.203930, Train accuracy: 0.1088
Val loss: 114.216644, Val accuracy: 0.1074

[INFO] Network evaluation ...


Test progress: 100%|█████████████████████████████████████████| 2/2 [00:01<00:00,  1.37it/s, loss=88]


Confusion matrix :
[[ 0 14  5  3  0  0]
 [ 0 27 16 14  2  1]
 [ 0 11  7 13  2  0]
 [ 0  5 11 22  2  2]
 [ 0  0  3 21  3  3]
 [ 0  1  2  3  3  5]]

MS: 0.0000

QWK: 0.5270

MAE: 0.9502

CCR: 0.3184

1-off: 0.7861

[INFO] Total training time: 61.98s
